In [3]:
from app import app, db
from app.models import Shapefile, shapefile_schema
import app.wof as wof
import geojson
from shapely.geometry import shape
from geoalchemy2.shape import to_shape
from sqlalchemy import text, select
import yaml

cities = yaml.safe_load(open('locality_configs.yml'))['reporting_market']

wof_ids = []

for city in cities:
    wof_ids.append(city['id'])
wof_ids = tuple(wof_ids)
wof_items = wof.from_ids(wof_ids)

with app.app_context() as session:
    query = select(Shapefile.id)
    existing_cities = db.session.scalars(query).all()
    
    for wof_item in wof_items:
        shapefiles = []
        if wof_item.id not in existing_cities:
            shapefiles.append(
                Shapefile(
                    id=wof_item.id,
                    name=wof_item.name,
                    placetype=wof_item.placetype,
                    parent=wof_item.parent,
                    geometry=shape(geojson.loads(wof_item.geojson).geometry).wkt,
                    bbox="".join(str(geojson.loads(wof_item.geojson).bbox))
                )
            )


OperationalError: (sqlite3.OperationalError) database is locked
(Background on this error at: https://sqlalche.me/e/20/e3q8)